In [3]:
from brian2 import *
import numpy as np
import warnings

#warnings.filterwarnings("ignore", category=UserWarning, module='brian2.codegen.generators.base')

start_scope()

defaultclock.dt = 0.0001*ms  

# Custom timing function
@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, result=1, sum=1, spikes_received=1)
def spike_timing(w, global_clock, layer, sum, spikes_received): 
    #print(global_clock)
    x = global_clock % 1
    if w >= 0:
        return (x ** (1 - w)) 
    else:
        return (1 - (1 - x) ** (1 + w)) 
    
@implementation('numpy', discard_units=True)
@check_units(layer=1, result=1, sum=1, spikes_received=1)
def math1(layer, sum, spikes_received): 
    return (sum/spikes_received )+ layer


In [4]:
# Urd, Verdande, Skuld 

# did not apply the specficyed weifhts to the first layer yet will maybe do after
def run_Urd(inputs, weights_1, weights_2, weights_3):
    '''4-10-3 SNN'''
    # will add check of weights # so it all works
    n_input = 4 
    n_hidden = 10
    n_output = 3
    n_total = n_input + n_hidden + n_output

    neurons = NeuronGroup(n_total, '''
        v : 1
        sum : 1
        spikes_received : 1
        scheduled_time : second
        global_clock : 1
    ''', threshold='v > 1', reset='v = 0', method='exact')
    neurons.v = 0
    neurons.scheduled_time = 1e9 * second
    neurons.global_clock = 0.0
    neurons.sum = 0.0
    neurons.spikes_received = 0.0


    indicess = [i for i in range(n_input)]
    stim = SpikeGeneratorGroup(n_input, indices=indicess, times=(inputs*ms))

    syn_input = Synapses(stim, neurons[0:n_input], '''
        w : 1
        layer : 1
    ''', on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
    ''')
    syn_input.connect(j='i')
    syn_input.w = weights_1
    syn_input.layer = 0

    syn_hidden = Synapses(neurons[0:n_input], neurons[n_input:n_input+n_hidden], '''
        w : 1
        layer : 1
    ''', on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
    ''')
    for inp in range(n_input):
        for hid in range(n_hidden):
            syn_hidden.connect(i=inp, j=hid)

    syn_hidden.w = weights_2
    syn_hidden.layer = 1


    syn_output = Synapses(
        neurons[n_input:n_input+n_hidden], 
        neurons[n_input+n_hidden:n_total], 
        '''
        w : 1
        layer : 1
        ''',
        on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
        '''
    )

    for hid in range(n_hidden):
        for out in range(n_output):
            syn_output.connect(i=hid, j=out)

    # Set weights in correct order
    syn_output.w[:] = weights_3
    syn_output.layer = 2

    print(syn_output.i[:], syn_output.j[:])
    #weights_into_output_1 = weights_3[1::3]



    neurons.run_regularly('''
        v = int(abs(t - scheduled_time) < 0.0005*ms) * 1.2
        global_clock += 0.001
    ''', dt=0.001*ms)


    spikemon = SpikeMonitor(neurons)

    run(5*ms)

    result = []

    for i in range(n_total):
        times = spikemon.spike_trains()[i]
        if len(times) > 0:
            result.append(round(times[0]/ms, 3))
        else:
            result.append(None)  # or some other placeholder like float('nan')
            
    return result


def calc_cost(outputs, desired_outputs):
    return 0.5 * ((outputs - desired_outputs) ** 2)


w_1 = [0] * 4 #np.random.uniform(  0.05, .95, size=4) #[0.5] * 4
w_2 = [0] * 40 #np.random.uniform( -.95, .95, size=40) #[0.5] * 40
w_3 = [0] * 30 #np.random.uniform( -.95, .95, size=30) #[0.5] * 30 



def update_weights_input_neuron(hidden_activations, weights, actual_output, desired_output, learning_rate=0.2):

    """
    Update all weights going into a single output neuron using gradient descent.

    Args:
        hidden_activations (np.ndarray): Activations from hidden neurons (shape: [n_hidden]).
        weights (np.ndarray): Current weights into the output neuron (shape: [n_hidden]).
        actual_output (float): Current output of this neuron.
        desired_output (float): Target output for this neuron.
        learning_rate (float): Learning rate.

    Returns:
        np.ndarray: Updated weights (shape: [n_hidden]).
    """
    error = actual_output - desired_output
    gradients = error * hidden_activations
    updated_weights = weights - learning_rate * gradients
    updated_weights = np.clip(updated_weights, -0.999999, 0.999999)
    return updated_weights


def update_single_hidden_neuron_weights(input_activations, weights_to_hidden, downstream_weights, output_errors, hidden_activation, learning_rate=0.2):
    """
    Updates the weights into a single hidden neuron.

    Args:
        input_activations (np.ndarray): Input activations (shape: [n_input]).
        weights_to_hidden (np.ndarray): Weights into this hidden neuron (shape: [n_input]).
        downstream_weights (np.ndarray): Weights from this hidden neuron to each output neuron (shape: [n_output]).
        output_errors (np.ndarray): Errors at each output neuron (shape: [n_output]).
        hidden_activation (float): Activation value of this hidden neuron.
        learning_rate (float): Learning rate.

    Returns:
        np.ndarray: Updated weights for this hidden neuron (shape: [n_input]).
    """
    propagated_error = np.dot(output_errors, downstream_weights)
    derivative = hidden_activation * (1 - hidden_activation)
    local_gradient = propagated_error * derivative
    gradients = local_gradient * input_activations
    updated_weights = weights_to_hidden - learning_rate * gradients
    updated_weights = np.clip(updated_weights, -0.999999, 0.999999)
    return updated_weights


def update_hidden_layer_weights(spike_times, output_errors):
    lr = 0.2

    input_activations = np.array(spike_times[:4])
    hidden_activations = np.array(spike_times[4:14])

    for i in range(10):  # 10 hidden neurons
        start_idx = i * 4
        end_idx = (i + 1) * 4

        weights_to_hidden_i = w_2[start_idx:end_idx]
        hidden_activation = hidden_activations[i]

        # FIXED: Correct indexing for downstream weights
        # Each hidden neuron i connects to all 3 outputs with weights at positions:
        # i*3, i*3+1, i*3+2
        downstream_weights = np.array([
            w_3[i * 3 + 0],  # weight from hidden neuron i to output 0
            w_3[i * 3 + 1],  # weight from hidden neuron i to output 1  
            w_3[i * 3 + 2],  # weight from hidden neuron i to output 2
        ])
        
        # Debug: Print the indices being used
        print(f"  Weight indices for neuron {i}: [{i*3}, {i*3+1}, {i*3+2}]")
        print(f"  w_3 values at those indices: {w_3[i*3:i*3+3]}")

        propagated_error = np.dot(output_errors, downstream_weights)
        derivative = hidden_activation * (1 - hidden_activation)
        local_gradient = propagated_error * derivative
        gradients = local_gradient * input_activations

        # print(f"\nHidden Neuron {i}")
        # print(f"  Hidden Activation: {hidden_activation}")
        # print(f"  Downstream Weights: {downstream_weights}")
        # print(f"  Propagated Error: {propagated_error}")
        # print(f"  Local Gradient: {local_gradient}")
        # print(f"  Gradients: {gradients}")

        updated_weights = weights_to_hidden_i - lr * gradients
        updated_weights = np.clip(updated_weights, -0.999999, 0.999999)
        w_2[start_idx:end_idx] = updated_weights

    # print("hidden layer weights updated")


def update_last_layer_weights(spike_times, desired_outputs):

    lr = 0.2 # will make more global later
    
    var = spike_times
    hidden_activations = np.array(var[4:14])


    weights_to_output_0 = w_3[0::3]
    weights_to_output_1 = w_3[1::3]
    weights_to_output_2 = w_3[2::3]

    actual_output_0 = var[-3]
    actual_output_1 = var[-2]
    actual_output_2 = var[-1]


    desired_output_0 = desired_outputs[0]
    desired_output_1 = desired_outputs[1]
    desired_output_2 = desired_outputs[2]


    new_weights_0 = update_weights_input_neuron(
        hidden_activations,
        weights_to_output_0,
        actual_output_0,
        desired_output_0,
        learning_rate=lr
    )

    new_weights_1 = update_weights_input_neuron(
        hidden_activations,
        weights_to_output_1,
        actual_output_1,
        desired_output_1,
        learning_rate=lr
    )

    new_weights_2 = update_weights_input_neuron(
        hidden_activations,
        weights_to_output_2,
        actual_output_2,
        desired_output_2,
        learning_rate=lr
    )

    w_3[0::3] = new_weights_0
    w_3[1::3] = new_weights_1
    w_3[2::3] = new_weights_2

    print("weights updated")



In [5]:

training_data = [[0.1, 0.5, 0.1, 0.75], [0.3, 0.5, 0.21, 0.65]]
result_data = [0, 1]

def train_Urd(training_data, result_data, loops):
    samples = len(training_data)

    wanted_output = [[-.1, -.1, -.1] for _ in range(samples)]
    
    for i in range(samples):
        if result_data[i] == 0:
            wanted_output[i] = [0.99, 0.0, 0.0]
        elif result_data[i] == 1:
            wanted_output[i] = [0.0, 0.99, 0.0]
        elif result_data[i] == 2:
            wanted_output[i] = [0.0, 0.0, 0.99]

    for j in range(loops):
        for i in range(samples):
            try:
                var = run_Urd(training_data[i], w_1, w_2, w_3)

                print(f"Iter {i} Output:", real_outputs(var))

                update_last_layer_weights(var, wanted_output[i])

                # calc error for hidden layer backprop
                output_errors = get_output_errors(var, wanted_output[i])

                update_hidden_layer_weights(var, output_errors)

            except Exception as e:
                print(f"Error in iteration {i}: {e}")

        print(f"{j + 1} loop of training data completed")
    
    print(f"{loops} loops completed")



train_Urd(training_data, result_data, 1)

WARNING    Cannot use Cython, a test compilation failed: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/ (DistutilsPlatformError) [brian2.codegen.runtime.cython_rt.cython_rt.failed_compile_test]
INFO       Cannot use compiled code, falling back to the numpy code generation target. Note that this will likely be slower than using compiled code. Set the code generation to numpy manually to avoid this message:
prefs.codegen.target = "numpy" [brian2.devices.device.codegen_fallback]


[0 0 0 1 1 1 2 2 2 3 3 3 4 4 4 5 5 5 6 6 6 7 7 7 8 8 8 9 9 9] [0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2]


WARNING    Came across an abstract code block that may not be well-defined: the outcome may depend on the order of execution. You can ignore this warning if you are sure that the order of operations does not matter. 3 lines of abstract code, first line is: 'spikes_received += 1 (in-place)'
 [brian2.codegen.generators.base]
INFO       Failed to vectorise code, falling back on Python loop: note that this will be very slow! Switch to another code generation target for best performance (e.g. cython). First line is: spikes_received += 1 (in-place) [brian2.codegen.generators.numpy_generator]
WARNING    Came across an abstract code block that may not be well-defined: the outcome may depend on the order of execution. You can ignore this warning if you are sure that the order of operations does not matter. 3 lines of abstract code, first line is: 'spikes_received += 1 (in-place)'
 [brian2.codegen.generators.base]
WARNING    Came across an abstract code block that may not be well-defined: the ou

Error in iteration 0: name 'real_outputs' is not defined


WARNING    Came across an abstract code block that may not be well-defined: the outcome may depend on the order of execution. You can ignore this warning if you are sure that the order of operations does not matter. 3 lines of abstract code, first line is: 'spikes_received += 1 (in-place)'
 [brian2.codegen.generators.base]
WARNING    Came across an abstract code block that may not be well-defined: the outcome may depend on the order of execution. You can ignore this warning if you are sure that the order of operations does not matter. 3 lines of abstract code, first line is: 'spikes_received += 1 (in-place)'
 [brian2.codegen.generators.base]
WARNING    Came across an abstract code block that may not be well-defined: the outcome may depend on the order of execution. You can ignore this warning if you are sure that the order of operations does not matter. 3 lines of abstract code, first line is: 'spikes_received += 1 (in-place)'
 [brian2.codegen.generators.base]


[0 0 0 1 1 1 2 2 2 3 3 3 4 4 4 5 5 5 6 6 6 7 7 7 8 8 8 9 9 9] [0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2]
Error in iteration 1: name 'real_outputs' is not defined
1 loop of training data completed
1 loops completed


In [ ]:
import numpy as np
import logging

logging.getLogger('brian2').setLevel(logging.ERROR)

inputs = [0.1, 0.2, 0.5, 0.9]
wanted_output = [2.1, 2.1, 2.9]

# Initialize weights if not already
# w_1 = np.random.uniform(0.05, 0.95, size=4)
# w_2 = np.random.uniform(-0.95, 0.95, size=40)
# w_3 = np.random.uniform(-0.95, 0.95, size=30)

def real_outputs(var):
    return [round(var[-3 + i] - 2, 4) for i in range(3)]

def get_output_errors(var, wanted_output):
    return np.array([var[-3 + i] - wanted_output[i] for i in range(3)])


print(w_2)
for i in range(5):
    try:
        var = run_Urd(inputs, w_1, w_2, w_3)

        print(f"Iter {i} Output:", real_outputs(var))

        update_last_layer_weights(var, wanted_output)

        # Compute error for hidden layer backprop
        output_errors = get_output_errors(var, wanted_output)

        update_hidden_layer_weights(var, output_errors)

    except Exception as e:
        print(f"Error in iteration {i}: {e}")

# Final output
print("Final Output:", real_outputs(var))
print(w_2)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0 0 0 1 1 1 2 2 2 3 3 3 4 4 4 5 5 5 6 6 6 7 7 7 8 8 8 9 9 9] [0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2]
Iter 0 Output: [0.428, 0.428, 0.428]
weights updated
  Weight indices for neuron 0: [0, 1, 2]
  w_3 values at those indices: [-0.09361119999999996, 0.02054880000000002, 0.13470880000000002]
  Weight indices for neuron 1: [3, 4, 5]
  w_3 values at those indices: [-0.09361119999999996, 0.02054880000000002, 0.13470880000000002]
  Weight indices for neuron 2: [6, 7, 8]
  w_3 values at those indices: [-0.09361119999999996, 0.02054880000000002, 0.13470880000000002]
  Weight indices for neuron 3: [9, 10, 11]
  w_3 values at those indices: [-0.09361119999999996, 0.02054880000000002, 0.13470880000000002]
  Weight indices for neuron 4: [12, 13, 14]
  w_3 values at those indices: [-0.09361119999999996, 0.02054880000000002, 0.13470880000000002]
  Weight ind

In [7]:
print(w_2)
list = [i % 10 for i in range(40)]
print(list)
for i in range(10):
    print(list[i::10])  

[-0.011241727353189564, -0.02237215047515943, -0.05576341984106902, -0.10028511232894849, -0.011151653233920672, -0.022192894059584703, -0.055316616536576796, -0.09948157983923292, -0.011232780566088193, -0.022354345483007194, -0.0557190402337642, -0.1002052999014402, -0.011151653233920672, -0.022192894059584703, -0.055316616536576796, -0.09948157983923292, -0.011241727353189564, -0.02237215047515943, -0.05576341984106902, -0.10028511232894849, -0.011151653233920672, -0.022192894059584703, -0.055316616536576796, -0.09948157983923292, -0.011232780566088193, -0.022354345483007194, -0.0557190402337642, -0.1002052999014402, -0.011151653233920672, -0.022192894059584703, -0.055316616536576796, -0.09948157983923292, -0.011241727353189564, -0.02237215047515943, -0.05576341984106902, -0.10028511232894849, -0.011151653233920672, -0.022192894059584703, -0.055316616536576796, -0.09948157983923292]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3